In [1]:
import time
import time, psutil, os
start = time.time()
m0 = psutil.Process(os.getpid()).memory_info().rss / 1024**2

In [2]:
import re
import unidecode
import polars as pl
import pandas as pd
import numpy as np
from rapidfuzz import fuzz
from pathlib import Path

# Moradas de Histórico

In [3]:
import polars as pl

ficheiro = Path.home() / "Downloads" / "06.BNE" / "addressGeoLoc_export_20221006.csv"

df = pl.read_csv(
    ficheiro,
    separator=",",
    quote_char='"',
    skip_rows=1,
    has_header=False,
    new_columns=["MORADA", "CP", "LOCALIDADE", "LATITUDE", "LONGITUDE"],
    decimal_comma=True
)

pl.Config.set_tbl_formatting("UTF8_FULL")  # formato mais limpo
pl.Config.set_tbl_rows(5)                 # nº de linhas a mostrar
pl.Config.set_tbl_width_chars(120) 
pl.Config.set_tbl_cols("10_000")    



polars.config.Config

In [4]:
df.head()

MORADA,CP,LOCALIDADE,LATITUDE,LONGITUDE
str,str,str,str,str
"""RUA CESARIO VERDE LOTE 3 A DAS…","""2660""","""FRIELAS""","""38.852697456""","""-9.16971532999997"""
"""RUA DO ROXICO NR50""","""3865-110""","""FERMELA""","""40.709808191""","""-8.54878568099997"""
"""AVENIDA 13 DE MAIO N 536""","""3885-227""","""CORTEGACA OVR""","""40.94109138""","""-8.61864986999996"""
"""AV REINALDO SANTOS N 24 3 DTO""","""2675-673""","""ODIVELAS""","""38.7926955990001""","""-9.18881750599996"""
"""RUA DO BARREIRO N 547""","""4405-730""","""VILA NOVA DE GAIA""","""41.113905909""","""-8.63062512299996"""


In [5]:
df_filtrado = df.filter(
    df["CP"].str.slice(0, 2).is_in(["40"])
)

df_filtrado

MORADA,CP,LOCALIDADE,LATITUDE,LONGITUDE
str,str,str,str,str
"""TRAVESSA DE SA O CARLOS 1 1 FR…","""4050-544""","""PORTO""","""41.15159""","""-8.614601"""
"""RUA DA FIRMEZA 482""","""4000-216""","""PORTO""","""41.1510188030001""","""-8.60600840199993"""
"""RUA DE CAMOES NO93 5OA""","""4000-144""","""PORTO""","""41.152648013""","""-8.61019293599998"""
…,…,…,…,…
"""R NOSSA SENHORA DE FATIMA 322""","""4050-426""","""PORTO""","""41.1594219950001""","""-8.62593197999996"""
"""RUA DE CAMOES 289 3D""","""4000""","""PORTO""","""41.15445""","""-8.610141"""


In [6]:
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    # converter para radianos
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    # fórmula haversine
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    R = 6371000  # raio da Terra em metros
    return R * c


# Moradas Base Nacional de Endereços (BNE) 

In [7]:

ficheiro = Path.home() / "Downloads" / "06.BNE" / "df_Principal_f.csv"

df_BNE = pl.read_csv(
    ficheiro,
    separator=",",
    quote_char='"',
    skip_rows=1,
    has_header=False,
    new_columns=["ART_COD",	"ART_TIPO","ART_TITULO","ART_DESIG","PORTA_NUM","CP4","CP3","CPALF","LOCALIDADE","LONG_PORTA","LAT_PORTA"],
    decimal_comma=True
)

In [8]:
df_BNE


ART_COD,ART_TIPO,ART_TITULO,ART_DESIG,PORTA_NUM,CP4,CP3,CPALF,LOCALIDADE,LONG_PORTA,LAT_PORTA
i64,str,str,str,i64,i64,i64,str,str,str,str
61213,"""Rua""",null,"""Abraos""",49,4000,12,"""PORTO""","""Porto""","""-8.59""","""41.15"""
61213,"""Rua""",null,"""Abraos""",32,4000,12,"""PORTO""","""Porto""","""-8.59""","""41.15"""
61213,"""Rua""",null,"""Abraos""",10,4000,12,"""PORTO""","""Porto""","""-8.59""","""41.15"""
…,…,…,…,…,…,…,…,…,…,…
2147140000,"""Beco""",null,"""Passos Manuel""",null,4000,7,"""PORTO""","""Porto""","""-8.6""","""41.14"""
2147140000,"""Beco""",null,"""Passos Manuel""",null,4000,7,"""PORTO""","""Porto""","""-8.6""","""41.14"""


In [9]:

# ==========================
# 2. Constantes
# ==========================
R = 6371000.0
p = np.pi / 180.0
limite_deg = 0.005  # ~500m

# ==========================
# 3. Função para limpar coordenadas
# ==========================
def clean_coord(col):
    # substitui vírgulas → ponto e converte para float
    return (
        pl.col(col)
          .cast(pl.Utf8)
          .str.replace(",", ".")
          .replace(["None","nan","NaN",""], None)
          .cast(pl.Float64)
    )

# ==========================
# 4. Preparar bases (lazy → eager)
# ==========================
df_portas = (
    df_filtrado
    .with_columns([
        pl.col("CP").str.slice(0, 2).alias("CP2"),
        clean_coord("LATITUDE").alias("LATITUDE"),
        clean_coord("LONGITUDE").alias("LONGITUDE")
    ])
    .drop_nulls(["LATITUDE", "LONGITUDE", "CP2"])
)

df_bne = (
    df_BNE
    .with_columns([
        pl.col("CP4").cast(pl.Utf8).str.slice(0, 2).alias("CP2"),
        clean_coord("LAT_PORTA").alias("LAT_PORTA"),
        clean_coord("LONG_PORTA").alias("LONG_PORTA")
    ])
    .filter(pl.col("CP2") == "40")  # <--- filtro coerente
    .drop_nulls(["LAT_PORTA", "LONG_PORTA", "CP2"])
)



In [10]:
df_portas


MORADA,CP,LOCALIDADE,LATITUDE,LONGITUDE,CP2
str,str,str,f64,f64,str
"""TRAVESSA DE SA O CARLOS 1 1 FR…","""4050-544""","""PORTO""",41.15159,-8.614601,"""40"""
"""RUA DA FIRMEZA 482""","""4000-216""","""PORTO""",41.151019,-8.606008,"""40"""
"""RUA DE CAMOES NO93 5OA""","""4000-144""","""PORTO""",41.152648,-8.610193,"""40"""
…,…,…,…,…,…
"""R NOSSA SENHORA DE FATIMA 322""","""4050-426""","""PORTO""",41.159422,-8.625932,"""40"""
"""RUA DE CAMOES 289 3D""","""4000""","""PORTO""",41.15445,-8.610141,"""40"""


In [11]:
# ==========================
# 5. Porta BNE – k=10 vizinhos mais próximos (<= 500 m)
# ==========================
import numpy as np
import polars as pl
from sklearn.neighbors import BallTree

# -------------------------------------------------
# Configurações
# -------------------------------------------------
R = 6371000.0
RAIO_M = 500.0
K_NEIGHBORS = 500   

# -------------------------------------------------
# 5.1 Coordenadas
# -------------------------------------------------
coords_portas = np.radians(df_portas.select(["LATITUDE", "LONGITUDE"]).to_numpy())
coords_bne = np.radians(df_bne.select(["LAT_PORTA", "LONG_PORTA"]).to_numpy())

# -------------------------------------------------
# 5.2 Busca k vizinhos
# -------------------------------------------------
tree = BallTree(coords_bne, metric="haversine")
distancias, indices = tree.query(coords_portas, k=K_NEIGHBORS)

# Flatten com reshape
dist_m = (distancias * R).flatten()
idx_bne = indices.flatten().astype(np.uint32)

# Índice da porta (repetido K vezes)
idx_porta = np.repeat(np.arange(len(df_portas), dtype=np.uint32), K_NEIGHBORS)

# -------------------------------------------------
# 5.3 Criar DataFrame + filtro
# -------------------------------------------------
pairs = (
    pl.DataFrame({
        "idx_porta": idx_porta,
        "idx_bne":   idx_bne,
        "dist_m":    dist_m
    })
    .filter(pl.col("dist_m") <= RAIO_M)
    .join(df_portas.with_row_count("idx_porta"), on="idx_porta", how="left")
    .join(df_bne.with_row_count("idx_bne"),    on="idx_bne",    how="left", suffix="_BNE")
    .drop(["idx_porta", "idx_bne"])
)

# -------------------------------------------------
# 6. Resultado
# -------------------------------------------------
print(f"Total de matches (k={K_NEIGHBORS}): {len(pairs)}")
pairs

/var/folders/8m/hh_3__ks6m12ytdrpdgv3zxc0000gn/T/ipykernel_20570/2419991662.py:44: DeprecationWarning: `DataFrame.with_row_count` is deprecated; use `with_row_index` instead. Note that the default column name has changed from 'row_nr' to 'index'.
  .join(df_portas.with_row_count("idx_porta"), on="idx_porta", how="left")
/var/folders/8m/hh_3__ks6m12ytdrpdgv3zxc0000gn/T/ipykernel_20570/2419991662.py:45: DeprecationWarning: `DataFrame.with_row_count` is deprecated; use `with_row_index` instead. Note that the default column name has changed from 'row_nr' to 'index'.
  .join(df_bne.with_row_count("idx_bne"),    on="idx_bne",    how="left", suffix="_BNE")


Total de matches (k=500): 10845312


dist_m,MORADA,CP,LOCALIDADE,LATITUDE,LONGITUDE,CP2,ART_COD,ART_TIPO,ART_TITULO,ART_DESIG,PORTA_NUM,CP4,CP3,CPALF,LOCALIDADE_BNE,LONG_PORTA,LAT_PORTA,CP2_BNE
f64,str,str,str,f64,f64,str,i64,str,str,str,i64,i64,i64,str,str,f64,f64,str
423.864204,"""TRAVESSA DE SA O CARLOS 1 1 FR…","""4050-544""","""PORTO""",41.15159,-8.614601,"""40""",3991213,"""Rua""",null,"""Cames""",295,4000,143,"""PORTO""","""Porto""",-8.61,41.15,"""40"""
423.864204,"""TRAVESSA DE SA O CARLOS 1 1 FR…","""4050-544""","""PORTO""",41.15159,-8.614601,"""40""",3991213,"""Rua""",null,"""Cames""",801,4000,140,"""PORTO""","""Porto""",-8.61,41.15,"""40"""
423.864204,"""TRAVESSA DE SA O CARLOS 1 1 FR…","""4050-544""","""PORTO""",41.15159,-8.614601,"""40""",9831213,"""Rua""",null,"""Gonalo Cristvo""",301,4000,267,"""PORTO""","""Porto""",-8.61,41.15,"""40"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
494.958229,"""RUA DE CAMOES 289 3D""","""4000""","""PORTO""",41.15445,-8.610141,"""40""",3991213,"""Rua""",null,"""Cames""",235,4000,143,"""PORTO""","""Porto""",-8.61,41.15,"""40"""
494.958229,"""RUA DE CAMOES 289 3D""","""4000""","""PORTO""",41.15445,-8.610141,"""40""",3991213,"""Rua""",null,"""Cames""",247,4000,143,"""PORTO""","""Porto""",-8.61,41.15,"""40"""


In [12]:
# ==========================
# 1. Imports + Dicionários
# ==========================
import polars as pl

abreviaturas = {
    "cmdt": "comandante", "cmte": "comandante",
    "dr": "doutor", "dra": "doutora",
    "sr": "senhor", "sra": "senhora", "srª": "senhora",
    "eng": "engenheiro", "enga": "engenheira", "engª": "engenheira",
    "prof": "professor", "profa": "professora", "profª": "professora",
    "arq": "arquiteto", "arqa": "arquiteta", "arqª": "arquiteta",
    "cap": "capitao", "maj": "major", "gen": "general",
    "ten": "tenente", "alm": "almirante",
    "sta": "santa", "sto": "santo", "s": "sao", "sao": "sao",
    "ns": "nossa senhora", "n s": "nossa senhora",
    "nsr": "nosso senhor", "n sr": "nosso senhor",
    "dom": "dom", "d": "dom",
    "visc": "visconde", "cond": "conde", "marq": "marques", "bar": "barao"
}

mapa_logradouros = {
    "r": "rua", "rua": "rua", "rª": "rua",
    "av": "avenida", "avd": "avenida", "avda": "avenida", "avenida": "avenida",
    "tv": "travessa", "trs": "travessa", "trav": "travessa", "travessa": "travessa",
    "pc": "praca", "pç": "praca", "pr": "praca", "prç": "praca", "praca": "praca",
    "lg": "largo", "al": "alameda", "bq": "beco",
    "esc": "escadas", "estr": "estrada", "cm": "caminho",
    "cç": "calcada", "qt": "quinta", "br": "bairro",
    "rot": "rotunda", "via": "via", "urb": "urbanizacao",
    "lgd": "lugar", "campo": "campo", "terreiro": "terreiro"
}

# ==========================
# 2. Função principal (100 % Polars)
# ==========================
def extrair_partes(df: pl.DataFrame) -> pl.DataFrame:
    # normalizar texto base
    df = df.with_columns([
        pl.col("MORADA")
          .str.to_lowercase()
          .str.replace_all(r"[\.]", "")
          .alias("morada_norm")
    ])

    # extrair componentes
    df = df.with_columns([
        # logradouro
        pl.col("morada_norm").str.extract(
            r"^(r|rua|av|avenida|tv|trav|travessa|pc|pr|praca|estr|estrada|beco|al|alameda|lg|largo|calcada|rotunda)", 1
        ).alias("logradouro"),

        # nome da rua
        pl.col("morada_norm").str.extract(
            r"^(?:r|rua|av|avenida|tv|trav|travessa|pc|pr|praca|estr|estrada|beco|al|alameda|lg|largo|calcada|rotunda)\s+([a-zçãõéóà\s]+?)(?:\d|$)", 
            1
        ).str.strip_chars().alias("nome_rua"),

        # número
        pl.col("morada_norm").str.extract(r"\s(\d+[a-z]?)\b", 1).alias("numero"),

        # andar
        pl.col("morada_norm").str.extract(r"(\d+º\s*[a-z]*|r/c)", 1).alias("andar"),

        # código postal
        pl.col("morada_norm").str.extract(r"(\d{4}-\d{3})", 1).alias("codigo_postal"),

        # localidade
        pl.when(pl.col("LOCALIDADE").is_not_null())
          .then(pl.col("LOCALIDADE").str.to_lowercase())
          .otherwise(pl.col("morada_norm").str.extract(r"([a-zçãõéó]+)$", 1))
          .alias("localidade")
    ])

    # normalizar logradouros
    expr = pl.col("logradouro")
    for k, v in mapa_logradouros.items():
        expr = expr.str.replace_all(rf"^{k}$", v)
    df = df.with_columns(expr.alias("logradouro_norm"))

    # limpar nome de rua
    expr_nome = pl.col("nome_rua")
    for log in set(mapa_logradouros.values()):
        expr_nome = expr_nome.str.replace_all(rf"\b{log}\b", "")
    expr_nome = (
        expr_nome
        .str.replace_all(r"\b(n|nr|nº|nro|numero|num)\b", "")
        .str.replace_all(r"\d+", "")
        .str.replace_all(r"\s+", " ")
        .str.strip_chars()
    )
    df = df.with_columns(expr_nome.alias("nome_rua_norm"))

    # normalizar abreviaturas (em Polars)
    expr_final = pl.col("nome_rua_norm")
    for k, v in abreviaturas.items():
        expr_final = expr_final.str.replace_all(rf"\b{k}\b", v)
    df = df.with_columns(expr_final.alias("nome_rua_final"))

    # normalizar restante
    df = df.with_columns([
        pl.col("numero").str.replace_all(r"[^0-9a-z]", "").alias("numero_norm"),
        pl.col("andar").str.replace_all(r"\s+", "").alias("andar_norm"),
        pl.col("codigo_postal").str.replace_all(r"\s+", "").alias("codigo_postal_norm"),
        pl.col("localidade").str.replace_all(r"\s+", "").alias("localidade_norm")
    ])

    return df


# SAE

In [13]:
# ==========================
# 1. Aplicar extrair_partes em Polars
# ==========================

# lado 1 → moradas originais
df1_parts = extrair_partes(
    pairs.select([
        pl.col("MORADA").alias("MORADA"),
        pl.col("LOCALIDADE").alias("LOCALIDADE")
    ])
)

# lado 2 → moradas do BNE
df2_parts = extrair_partes(
    pairs.select([
        pl.col("ART_DESIG").alias("MORADA"),
        pl.col("LOCALIDADE_BNE").alias("LOCALIDADE")
    ])
)


In [14]:
df2_parts.count()

MORADA,LOCALIDADE,morada_norm,logradouro,nome_rua,numero,andar,codigo_postal,localidade,logradouro_norm,nome_rua_norm,nome_rua_final,numero_norm,andar_norm,codigo_postal_norm,localidade_norm
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
10845312,10845312,10845312,4146120,0,0,0,0,10845312,4146120,0,0,0,0,0,10845312


In [15]:
df1_parts.count()

MORADA,LOCALIDADE,morada_norm,logradouro,nome_rua,numero,andar,codigo_postal,localidade,logradouro_norm,nome_rua_norm,nome_rua_final,numero_norm,andar_norm,codigo_postal_norm,localidade_norm
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
10845312,10845312,10845312,10546551,10491084,10544986,0,0,10845312,10546551,10491084,10491084,10544986,0,0,10845312


In [16]:
# ==========================
# 1. Imports
# ==========================
import polars as pl
from rapidfuzz import fuzz
import re, unidecode

# ==========================
# 2. Função: normalizar e extrair partes
# ==========================
def extrair_partes(df: pl.DataFrame) -> pl.DataFrame:
    abreviaturas = {
        "cmdt": "comandante", "cmte": "comandante",
        "dr": "doutor", "dra": "doutora",
        "sr": "senhor", "sra": "senhora", "srª": "senhora",
        "eng": "engenheiro", "enga": "engenheira", "engª": "engenheira",
        "prof": "professor", "profa": "professora", "profª": "professora",
        "arq": "arquiteto", "arqa": "arquiteta", "arqª": "arquiteta",
        "cap": "capitao", "maj": "major", "gen": "general",
        "ten": "tenente", "alm": "almirante",
        "sta": "santa", "sto": "santo", "s": "sao", "sao": "sao",
        "ns": "nossa senhora", "n s": "nossa senhora",
        "nsr": "nosso senhor", "n sr": "nosso senhor",
        "dom": "dom", "d": "dom"
    }

    mapa_logradouros = {
        "r": "rua", "av": "avenida", "tv": "travessa", "pc": "praca",
        "lg": "largo", "al": "alameda", "bq": "beco", "estr": "estrada",
        "cm": "caminho", "cç": "calcada", "rot": "rotunda"
    }

    def normalizar(txt):
        if not txt:
            return ""
        txt = unidecode.unidecode(str(txt).lower()).replace(".", "").strip()
        palavras = [abreviaturas.get(p, mapa_logradouros.get(p, p)) for p in txt.split()]
        return " ".join(palavras)

    def limpar_nome_rua(txt):
        if not txt:
            return ""
        txt = unidecode.unidecode(str(txt).lower())
        for log in set(mapa_logradouros.values()):
            txt = re.sub(rf"\b{log}\b", "", txt)
        txt = re.sub(r"\b(n|nr|nº|nro|numero|num)\b", "", txt)
        txt = re.sub(r"\d+", "", txt)
        palavras = txt.split()
        palavras = [abreviaturas.get(p, p) for p in palavras]
        txt = " ".join(palavras)
        return re.sub(r"\s+", " ", txt).strip()

    df_parts = df.with_columns([
        pl.col("MORADA").str.extract(
            r"(?i)^(R\.?|RUA|AV\.?|AVENIDA|TRAV\.?|TRAVESSA|PRAÇA|ESTRADA|BECO|ALAMEDA|LARGO|CALÇADA|ROTUNDA)",
            1).alias("logradouro"),
        pl.col("MORADA").str.extract(
            r"(?i)^(?:R\.?|RUA|AV\.?|AVENIDA|TRAV\.?|TRAVESSA|PRAÇA|ESTRADA|BECO|ALAMEDA|LARGO|CALÇADA|ROTUNDA)\s+([A-ZÇÃÕÉÓÀ\s]+?)(?:\d|$)",
            1).alias("nome_rua"),
        pl.col("MORADA").str.extract(r"\s(\d+[A-Z]?)\b", 1).alias("numero"),
        pl.col("MORADA").str.extract(r"(\d+º\s*[A-Z]*|R/C)", 1).alias("andar"),
        pl.col("MORADA").str.extract(r"(\d{4}-\d{3})", 1).alias("codigo_postal"),
        pl.when(pl.col("LOCALIDADE").is_not_null())
          .then(pl.col("LOCALIDADE"))
          .otherwise(pl.col("MORADA").str.extract(r"([A-ZÇÃÕÉÓ]+)$", 1))
          .alias("localidade")
    ])

    df_parts = df_parts.with_columns([
        pl.col("logradouro").map_elements(normalizar, return_dtype=pl.Utf8).alias("logradouro_norm"),
        pl.col("nome_rua").map_elements(limpar_nome_rua, return_dtype=pl.Utf8).alias("nome_rua_norm"),
        pl.col("numero").map_elements(normalizar, return_dtype=pl.Utf8).alias("numero_norm"),
        pl.col("localidade").map_elements(normalizar, return_dtype=pl.Utf8).alias("localidade_norm")
    ])

    return df_parts


# ==========================
# 3. Matching otimizado (sem loops duplos)
# ==========================
def calcular_fuzzy(row) -> float:
    # cálculo único linha a linha
    score_log = fuzz.token_sort_ratio(row["logradouro_norm"], row["logradouro_norm_BNE"])
    score_rua = fuzz.token_sort_ratio(row["nome_rua_norm"], row["nome_rua_norm_BNE"])
    score_num = fuzz.token_sort_ratio(str(row["numero_norm"]), str(row["numero_norm_BNE"]))
    score_loc = fuzz.token_sort_ratio(row["localidade_norm"], row["localidade_norm_BNE"])
    return 0.3*score_log + 0.4*score_rua + 0.2*score_num + 0.1*score_loc


# ==========================
# 4. Pipeline otimizado
# ==========================

# 4.1 garantir ID global
pairs = pairs.drop("id_pair", strict=False).with_row_index("id_pair")

# 4.2 normalizar moradas de ambos os lados (já emparelhadas)
df_parts = extrair_partes(
    pairs.select(["id_pair", "MORADA", "LOCALIDADE"])
)

df_bne_parts = extrair_partes(
    pairs.select(["id_pair", "ART_TIPO", "ART_DESIG", "LOCALIDADE_BNE"])
    .with_columns((pl.col("ART_TIPO") + " " + pl.col("ART_DESIG")).alias("MORADA"))
    .rename({"LOCALIDADE_BNE": "LOCALIDADE"})
)

# 4.3 juntar lado a lado
pairs_fuzzy = (
    df_parts
    .join(df_bne_parts, on="id_pair", how="inner", suffix="_BNE")
)

# 4.4 calcular score fuzzy linha a linha
pairs_fuzzy = pairs_fuzzy.with_columns(
    pl.struct(pairs_fuzzy.columns).map_elements(calcular_fuzzy, return_dtype=pl.Float64).alias("score")
)

# 4.5 aplicar threshold
df_matches = pairs_fuzzy.filter(pl.col("score") >= 85)

# ==========================
# 5. Output final
# ==========================
df_matches


id_pair,MORADA,LOCALIDADE,logradouro,nome_rua,numero,andar,codigo_postal,localidade,logradouro_norm,nome_rua_norm,numero_norm,localidade_norm,ART_TIPO,ART_DESIG,LOCALIDADE_BNE,MORADA_BNE,logradouro_BNE,nome_rua_BNE,numero_BNE,andar_BNE,codigo_postal_BNE,localidade_BNE,logradouro_norm_BNE,nome_rua_norm_BNE,numero_norm_BNE,localidade_norm_BNE,score
u32,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64
80770,"""RUA ALEGRIA NR712 RC DOMINGOS …","""PORTO""","""R""","""ALEGRIA NR""",null,null,null,"""PORTO""","""rua""","""alegria""",null,"""porto""","""Rua""","""Alegria""","""Porto""","""Rua Alegria""","""R""","""Alegria""",null,null,null,"""Porto""","""rua""","""alegria""",null,"""porto""",100.0
80772,"""RUA ALEGRIA NR712 RC DOMINGOS …","""PORTO""","""R""","""ALEGRIA NR""",null,null,null,"""PORTO""","""rua""","""alegria""",null,"""porto""","""Rua""","""Alegria""","""Porto""","""Rua Alegria""","""R""","""Alegria""",null,null,null,"""Porto""","""rua""","""alegria""",null,"""porto""",100.0
80774,"""RUA ALEGRIA NR712 RC DOMINGOS …","""PORTO""","""R""","""ALEGRIA NR""",null,null,null,"""PORTO""","""rua""","""alegria""",null,"""porto""","""Rua""","""Alegria""","""Porto""","""Rua Alegria""","""R""","""Alegria""",null,null,null,"""Porto""","""rua""","""alegria""",null,"""porto""",100.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
10844235,"""RUA GONCALO CRISTOVAO N371 N37…","""PORTO""","""R""","""GONCALO CRISTOVAO N""",null,null,null,"""PORTO""","""rua""","""goncalo cristovao""",null,"""porto""","""Rua""","""Gonalo Cristvo""","""Porto""","""Rua Gonalo Cristvo""","""R""","""Gonalo Cristvo""",null,null,null,"""Porto""","""rua""","""gonalo cristvo""",null,"""porto""",96.129032
10844236,"""RUA GONCALO CRISTOVAO N371 N37…","""PORTO""","""R""","""GONCALO CRISTOVAO N""",null,null,null,"""PORTO""","""rua""","""goncalo cristovao""",null,"""porto""","""Rua""","""Gonalo Cristvo""","""Porto""","""Rua Gonalo Cristvo""","""R""","""Gonalo Cristvo""",null,null,null,"""Porto""","""rua""","""gonalo cristvo""",null,"""porto""",96.129032


In [17]:
m1 = psutil.Process(os.getpid()).memory_info().rss / 1024**2
print("Tempo Polars:", round(time.time() - start, 2), "segundos")
print("Memória Polars:", round(m1 - m0, 2), "MB")

Tempo Polars: 297.17 segundos
Memória Polars: 2561.66 MB
